# Credit model with Azure Service and scikit-learn

### Contents

 1. Setup
 1. Test the sample model locally
 1. Register the model to your Azure workspace
 1. Deploy the model as Webservice
 1. Test the deployed model

**Note:** 1. This notebook works correctly with kernel **`IBM Runtime 22.1 on Python 3.9 XS`** if using IBM Watson Studio or else use standard Python 3.9 runtime..
          2. Sample model was created using scikit-learn version: `1.2.2`. Model predicts if there is a risk for credit.

## 1. Setup

### Install required packages

In [ ]:
# Install these packages if you are not using IBM Watson Studio
#!pip install --upgrade scikit-learn==1.2.2 | tail -n 1
#!pip install pandas
#!pip install --upgrade azureml-sdk

In [1]:
import warnings
warnings.filterwarnings('ignore')
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [ ]:
!pip install --upgrade azureml-core  | tail -n 1

### Action: Restart the kernel

### Get sample model

In [2]:
!rm -rf german_credit_risk_122.joblib
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/Azure/assets/models/credit_risk/german_credit_risk_122.joblib

--2024-07-24 12:03:23--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/Azure/assets/models/credit_risk/german_credit_risk_122.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 186444 (182K) [application/octet-stream]
Saving to: ‘german_credit_risk_122.joblib’

german_credit_risk_ 100%[===================>] 182.07K  --.-KB/s    in 0.06s   

2024-07-24 12:03:23 (2.86 MB/s) - ‘german_credit_risk_122.joblib’ saved [186444/186444]



### Load model from local file

In [3]:
import os
import joblib

In [4]:
model_name = "safetodelgerman_credit_risk"
model_path = "german_credit_risk_122.joblib" #"german_credit_risk.joblib"

clf = joblib.load(os.path.join(os.getcwd(), model_path))

## 2. Test the sample model locally

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

### Load test data

In [6]:
!rm -rf credit_risk_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/credit_risk_training.csv

--2024-07-24 12:03:34--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/credit_risk_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

credit_risk_trainin 100%[===================>] 677.95K  --.-KB/s    in 0.1s    

2024-07-24 12:03:35 (5.47 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



In [7]:
data_df = pd.read_csv("credit_risk_training.csv",dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 
                              'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})
test_data = data_df
test_data = test_data.drop('Risk', axis=1)

### Predict on test data

In [8]:
predictions = clf.predict(test_data)

print("Model accuracy: {:.3f}".format(accuracy_score(data_df.Risk, predictions)))

Model accuracy: 0.812


## 3. Register the model to your Azure workspace

In [9]:
import os
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.authentication import InteractiveLoginAuthentication

Please provide your Azure ML Service credentials.

In [10]:
az_ml_service_credentials = {'tenant_id': '***',
                             'subscription_id': '***', 
                             'resource_group': '***',
                             'workspace_name': '***'}

In [11]:
interactive_auth = InteractiveLoginAuthentication(tenant_id=az_ml_service_credentials['tenant_id'])

ws = Workspace(subscription_id=az_ml_service_credentials['subscription_id'], 
               resource_group=az_ml_service_credentials['resource_group'], 
               workspace_name=az_ml_service_credentials['workspace_name'], 
               auth=interactive_auth)

In [12]:
ws.write_config()

In [13]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: bias_dev
Azure region: southcentralus
Subscription id: 744bca72-2299-451c-b682-ed6fb75fb671
Resource group: bias_explainability


In [14]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = '***' # use your model deployment name
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    print('scoring endpoint', service.name)

scoring endpoint safetodeletegcr
scoring endpoint regression-boston
scoring endpoint binary-gcr-endpoint
scoring endpoint gcr-binary-endpoint
scoring endpoint gosales-multiclass-endpoint
scoring endpoint insurancefraud-binary
scoring endpoint adult-census-copy
scoring endpoint walking-activity-multiclass
scoring endpoint gcrnewfeb2127-1-gcr-without-scoring
scoring endpoint bostonregressio50-1-boston-regression
scoring endpoint gosales-final-custom-gosales-multiclass
scoring endpoint div-new-1-custom-gcr-final


In [15]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=ws,
                       model_name=model_name,                # Name of the registered model in your workspace.
                       model_path=model_path,                # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version='1.2.2',             # Version of scikit-learn used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='credit risk sample scikit model from Watson Studio',
                       tags={'area': 'german_credit_risk', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model safetodelgerman_credit_risk
Name: safetodelgerman_credit_risk
Version: 2


## 4. Deploy the model as Webservice

Model deploymen consist of following steps:

 - Create scoring script
 - Create environment file
 - Create configuration file
 - Deploy model in ACI

### Create scoring script

Scoring script must include two required functions:

 - The `init()` function, which typically loads the model into a global object. This function is run only once.
 - The `run(input_data)` function uses the model to predict a value based on the input data.

In [16]:
%%writefile score_azure.py
import os
import json
import numpy as np
import pandas as pd
import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('***') # use the model name here
    model = joblib.load(model_path)

def run(input_data):
    try:
        if type(input_data) is str:
            dict_data = json.loads(input_data)
        else:
            dict_data = input_data
            
        data = pd.DataFrame.from_dict(dict_data['input'])   
        predictions = model.predict(data)
        scores = model.predict_proba(data).tolist()
        records = []
        
        for pred, prob in zip(predictions, scores):
            records.append({"Scored Labels": pred, "Scored Probabilities": prob})
 
        result = {'output': records}
        
        return json.dumps(result)
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})

Overwriting score_azure.py


### Create environment file

Create an environment file that specifies all of the script's package dependencies.

In [17]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment

environment = Environment('my-sklearn-environment')

environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'joblib',
    'numpy',
    'pandas',
    'scikit-learn==1.2.2'
])

### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container.

In [18]:
from azureml.core.webservice import AciWebservice


aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=0.5
                                                ,tags={"data": "german_credit_risk",  "method" : "sklearn"}, 
                                               description='Predict Credit Risk with sklearn from Watson Studio')

### Deploy in ACI

Configure the image and deploy it with following steps:

 - Build an image
 - Register that image under the workspace
 - Send the image to the ACI container
 - Start up a container in ACI using the image
 - Get the web service HTTP endpoint

**Note:** Estimated time to complete: **about 7-8 minutes**.

In [19]:
%%time
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.model import InferenceConfig
from azureml.core.image import ContainerImage


service_name = '***' # deployment service name

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score_azure.py', environment=environment)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Running
2024-07-24 06:31:53+00:00 Check and wait for operation (6ba436c0-6d81-48f6-b493-7d17242eb2c0) to finish.
2024-07-24 06:31:56+00:00 Deleting service entity.
Succeeded


In [20]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = service_name
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    if service.name == credit_risk_deployment_name:
        credit_risk_scoring_endpoint = service.scoring_uri
        
print('scoring endpoint', credit_risk_scoring_endpoint)

scoring endpoint http://48bbe5ba-fad3-4ea9-a044-1ab2d382234f.southcentralus.azurecontainer.io/score


## 5. Test deployed model

### Prepare scoring payload

In [21]:
scoring_data = {"input":[
                {
                "CheckingStatus": "0_to_200", "LoanDuration": 31, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "other",
                "LoanAmount": 1889, "ExistingSavings": "100_to_500", "EmploymentDuration": "less_1", "InstallmentPercent": 3, "Sex": "female",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 32, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 1, "Job": "skilled", "Dependents": 1, "Telephone": "none", "ForeignWorker": "yes",
                },
                {
                "CheckingStatus": "no_checking", "LoanDuration": 13, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "car_new",
                "LoanAmount": 1389, "ExistingSavings": "100_to_500", "EmploymentDuration": "1_to_4", "InstallmentPercent": 2, "Sex": "male",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 25, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 2, "Job": "skilled", "Dependents": 2, "Telephone": "none", "ForeignWorker": "yes",
                }]}

### Send scoring request using an HTTP request

In [22]:
import requests
headers = {"Content-Type": "application/json"}
resp = requests.post(credit_risk_scoring_endpoint, json=scoring_data, headers=headers)
resp.json()

{'output': [{'Scored Labels': 'No Risk', 'Scored Probabilities': [0.8922524675865824, 0.10774753241341757]}, {'Scored Labels': 'No Risk', 'Scored Probabilities': [0.8335192848546905, 0.1664807151453095]}]}